In [1]:
import pandas as pd
import numpy as np

In [2]:
#sizes = {'real': {'train': 443757, 'val': 214354}, 'abstract': {'train': 60000, 'val': 30000}}

In [3]:
def get_summary(scenario, _set = 'val'):
    details_train = pd.read_csv('./data/predictions_details_{}_train'.format(scenario), header=None)
    details_train['_set'] = 'train'
    details_val = pd.read_csv('./data/predictions_details_{}_val'.format(scenario), header=None)
    details_val['_set'] = 'val'
    details = pd.concat([details_train, details_val])
    details[2] = details[2].astype(int)
    details.sort_values(by=2, inplace=True)

    prediction = pd.read_csv('./concatenated_data/{}_prediction.csv'.format(scenario))
    prediction.rename(columns={'2049':'q_type', '2048':'q_id'}, inplace=True)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    prediction['_set'] = details.set_index(2)[['_set']].loc[prediction['q_id'].astype(int)].values
    prediction['my_acc'] = (prediction.my_predicted == prediction.my_target).astype(int)
    prediction['base_acc'] = (prediction.base_predicted == prediction.base_target).astype(int)
    sset_prediction = prediction[prediction._set == _set]
    accs = (sset_prediction[['my_acc','base_acc']].sum()/sset_prediction.shape[0]).values
    print('acc', accs, accs[0] > accs[1])
    accurracies = sset_prediction[['q_type','my_acc','base_acc',
                              'sum_top_down', 'sum_bottom_up']].groupby(['q_type']).sum().reset_index()
    accurracies = accurracies.set_index('q_type')
    
    
    new_summary = pd.read_csv('./concatenated_data/queries_summary_{}.csv'.format(scenario))
    del new_summary['count_top_down']
    new_summary.rename(columns={'2049':'_q_type'}, inplace=True)
    
    
    new_summary = new_summary[new_summary._q_type.isin(accurracies.index.values)]
    new_summary['q_type'] = accurracies.loc[new_summary._q_type.values].index.values
    new_summary['my_acc'] = accurracies.loc[new_summary._q_type.values].my_acc.values
    new_summary['base_acc'] =  accurracies.loc[new_summary._q_type.values].base_acc.values
    val_count = pd.DataFrame(sset_prediction.q_type.value_counts())
    new_summary['_count'] = val_count.loc[new_summary._q_type.values].values
    new_summary['above_median'] = new_summary['_count'] > float(new_summary['_count'].median())
    new_summary['sum_top_down'] = accurracies.loc[new_summary._q_type.values].sum_top_down.values
    new_summary['sum_bottom_up'] =  accurracies.loc[new_summary._q_type.values].sum_bottom_up.values
    new_summary['winner'] = np.array(['top_down',
                                      'bottom_up'])[(new_summary.sum_bottom_up 
                                       > new_summary.sum_top_down).astype(int)]
    _sum = new_summary[['my_acc','base_acc']].sum()
    print('acc_sum', _sum.values, _sum[0] > _sum[1])#/accurracies.shape[0]
    assert (new_summary._q_type != new_summary.q_type).sum() == 0
    _sum = (new_summary.sum_top_down + new_summary.sum_bottom_up).values
    new_summary['sum_top_down'] = new_summary['sum_top_down']/_sum
    new_summary['sum_bottom_up'] = new_summary['sum_bottom_up']/_sum
    new_summary['_diff'] = (new_summary.sum_top_down - new_summary.sum_bottom_up)
    del new_summary['diff']
    del new_summary['q_type']
    del new_summary['sum_top_down']

    print(new_summary._count.sum())
    return new_summary

In [106]:
def get_differents(p=1):
    mean_dif = summary._diff.mean()
    std_dif = summary._diff.std()
    data_dif = summary._diff
    data_diff = pd.DataFrame( ((data_dif < mean_dif - p * std_dif).astype(int) + 
                             (data_dif > mean_dif + p * std_dif).astype(int)) > 0 )
    significantly_different = summary.loc[data_diff[data_diff._diff].index]
    print(data_diff.shape)
    different = significantly_different[significantly_different._count>2].sort_values(by='winner')
    
    print(np.round(float(different.shape[0]) / data_diff.shape[0] * 100, 3),'% query_types')
    print(np.round(float(different._count.sum())/ summary._count.sum() * 100, 3),'% queries')

    different['my_minus_base'] = different.my_acc - different.base_acc
    vals = different[['my_acc','base_acc']].sum().values
    print(vals, vals[0]>vals[1])
    print(different.my_minus_base.sum(), 'abs diff')
    print(different.shape[0],'len')
    return different

In [129]:
scenario = 'abstract'
summary = get_summary(scenario)
#p = 1.31 p = 1.38 p = 1
print('\n')
different = get_differents(1)

('acc', array([ 0.6383    ,  0.63913333]), False)
('acc_sum', array([19149, 19174]), False)
30000


(177, 1)
(12.994, '% query_types')
(21.766999999999999, '% queries')
(array([3555, 3487]), True)
(68, 'abs diff')
(23, 'len')


In [130]:
different

,_q_type,sum_bottom_up,count_bottom_up,_count,over_median,winner,my_acc,base_acc,above_median,_diff,my_minus_base
7,are there any,0.520157,0.5,172,True,bottom_up,148,149,True,-0.040314,-1
164,what color is,0.506299,0.5,81,True,bottom_up,69,71,True,-0.012599,-2
211,will it,0.506267,0.5,15,True,bottom_up,12,12,True,-0.012534,0
144,to walk,0.510516,0.5,4,False,bottom_up,3,3,False,-0.021033,0
86,is anyone,0.515345,0.5,95,True,bottom_up,88,89,True,-0.030691,-1
227,would a,0.506128,0.5,3,True,bottom_up,0,1,False,-0.012255,-1
63,have her,0.514602,0.5,8,True,bottom_up,4,5,False,-0.029203,-1
72,how many bushes,0.462417,0.5,87,True,top_down,44,36,True,0.075166,8
199,who is,0.464034,0.5,391,True,top_down,176,182,True,0.071933,-6
198,who,0.467151,0.5,137,True,top_down,56,61,True,0.065699,-5
